In [6]:
import math
import pandas as pd
import copy
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from prettytable import MARKDOWN
from numpy.random import normal
import numpy as np
import matplotlib

In [7]:
file = "26kLongRun.txt"
df = pd.read_csv(file, sep=',')



In [8]:
df.head(5)

,time,latitude,longitude,elevation,distance,cumulative_distance,duration,cumulative_duration,pace_metric
0,2024-03-31 00:00:00+00:00,59.317436,18.050744,22.0,0.000000,0.000000,0.0,0.0,6.711367
1,2024-03-31 00:00:00+00:00,59.317436,18.050744,22.0,0.000000,0.000000,0.0,0.0,6.711367
2,2024-03-31 00:00:01+00:00,59.317442,18.050786,22.0,2.483349,2.483349,1.0,1.0,6.711367
3,2024-03-31 00:00:04+00:00,59.317375,18.050818,22.0,7.683068,10.166417,3.0,4.0,6.507817
4,2024-03-31 00:00:06+00:00,59.317321,18.050844,21.0,6.195157,16.361574,2.0,6.0,5.380547


In [10]:
df.sort_values(by=["time"], inplace=True)

In [11]:
# Here, we get the unique trajectories
unique_uids = df.drop_duplicates(["time"])
unique_uids = unique_uids["time"].to_list()


In [12]:
class Point:
    R = 6371 # Radius of the earth
    def __init__(self, latitude, longitude, t):
        self.latitude = latitude
        self.longitude = longitude
        self.t = t

    def __str__(self):
        return "Point({},{},{})".format(self.latitude, self.longitude, self.t)

    def getDistance(self, point2):
        delta_lambda = math.radians(point2.latitude - self.latitude)
        delta_phi = math.radians(point2.longitude - self.longitude)
        a = math.sin(delta_lambda / 2) * math.sin(delta_lambda / 2) + math.cos(math.radians(self.latitude)) \
            * math.cos(math.radians(point2.latitude)) * math.sin(delta_phi / 2) * math.sin(delta_phi / 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = Point.R * c
        return distance

In [13]:
class Trajectory:
    def __init__(self, points = []):
        self.points = points

    def addNoise(self, mu, sigma):
        return Trajectory([Point(i.latitude + normal(mu, sigma), i.longitude + normal(mu, sigma), i.t) for i in self.points])


In [15]:
# Then, we split the trajectories
points = []
trajectories = []
for i in range(len(unique_uids)):
    #for i in range(1):
    tmp_df = df.loc[df["time"]==unique_uids[i]]
    len(tmp_df)
    for j in range(len(tmp_df)):
        points.append(Point(tmp_df.iloc[j][0], tmp_df.iloc[j][1], tmp_df.iloc[j][2]))
    trajectories.append(copy.deepcopy(points))
    points.clear()


/tmp/ipykernel_76175/62256649.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  points.append(Point(tmp_df.iloc[j][0], tmp_df.iloc[j][1], tmp_df.iloc[j][2]))


In [16]:
len(trajectories)

665

In [17]:
P = Trajectory(trajectories[0])

In [68]:
import skmob
tdf = skmob.TrajDataFrame.from_file('26kLongRun.txt', datetime='datetime', latitude='lat', longitude='lon')
# print a portion of the TrajDataFrame
print(tdf.head())



#Q = P.addNoise(0, 0.0005)

                        time   latitude  longitude  elevation  distance  \
0  2024-03-31 00:00:00+00:00  59.317436  18.050744       22.0  0.000000   
1  2024-03-31 00:00:00+00:00  59.317436  18.050744       22.0  0.000000   
2  2024-03-31 00:00:01+00:00  59.317442  18.050786       22.0  2.483349   
3  2024-03-31 00:00:04+00:00  59.317375  18.050818       22.0  7.683068   
4  2024-03-31 00:00:06+00:00  59.317321  18.050844       21.0  6.195157   

   cumulative_distance  duration  cumulative_duration  pace_metric  
0             0.000000       0.0                  0.0     6.711367  
1             0.000000       0.0                  0.0     6.711367  
2             2.483349       1.0                  1.0     6.711367  
3            10.166417       3.0                  4.0     6.507817  
4            16.361574       2.0                  6.0     5.380547  


In [69]:

data_df = pd.DataFrame(tdf, columns=['latitude', 'longitude', 'time', 'elevation', 'pace_metric'])
# print the type of the object
print(type(data_df))

<class 'pandas.core.frame.DataFrame'>


In [70]:
tdf = skmob.TrajDataFrame(data_df, latitude='latitude', longitude='longitude', datetime='time')
# print the type of the object
print(type(tdf))

<class 'skmob.core.trajectorydataframe.TrajDataFrame'>


In [71]:

print(tdf.head())


         lat        lng                  datetime  elevation  pace_metric
0  59.317436  18.050744 2024-03-31 00:00:00+00:00       22.0     6.711367
1  59.317436  18.050744 2024-03-31 00:00:00+00:00       22.0     6.711367
2  59.317442  18.050786 2024-03-31 00:00:01+00:00       22.0     6.711367
3  59.317375  18.050818 2024-03-31 00:00:04+00:00       22.0     6.507817
4  59.317321  18.050844 2024-03-31 00:00:06+00:00       21.0     5.380547


In [72]:
# Trying to grip the syntax.
tdf.plot_trajectory(zoom=12, weight=3, opacity=0.9, max_points=None, max_users=None)

/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/skmob/core/trajectorydataframe.py:569: UserWarning: Only the trajectories of the first 10 users will be plotted. Use the argument `max_users` to specify the desired number of users, or filter the TrajDataFrame.
  return plot.plot_trajectory(self, map_f=map_f, max_users=max_users, max_points=max_points, style_function=style_function,
